In [31]:
import ee
from geemap import geemap
import pandas as pd
import ee
import datetime
import intake
from pathlib import Path
import numpy as np

In [7]:
ee.Authenticate()
ee.Initialize(project='sentinel-treeclassification')

In [9]:
catalog = intake.open_catalog(Path('catalog.yml'))
source = getattr(catalog, 'treesat')
gdf = source.read()[source.metadata['usecols']]
# Geopandas does not seem to always recognise the crs.
gdf.crs = 25832
gdf = gdf.to_crs(epsg=4326)
# Add some padding to avoid border polygons being cut off.
bbox = ee.Geometry.BBox(*(gdf.geometry.total_bounds + 0.01))

In [53]:
class SentinelGetter:
    def mask_s2_clouds(self, image):
      # Quality assessment with resolution in meters
      qa = image.select('QA60')
      # Bits 10 and 11 are clouds and cirrus, respectively.
      cloud_bit_mask = 1 << 10
      cirrus_bit_mask = 1 << 11
      # Both flags should be set to zero, indicating clear conditions.
      mask = (
          qa.bitwiseAnd(cloud_bit_mask)
          .eq(0)
          .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
      )
      return image.updateMask(mask)

    def get_image(self, start_date, end_date):
        selected_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12', 'TCI_R', 'TCI_G', 'TCI_B']
        image = (
            ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
            .filterDate(start_date, end_date)
            # Pre-filter to get less cloudy granules.
            # .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
            .map(self.mask_s2_clouds)
            .select(selected_bands)
        )
        return image

start_date = datetime.datetime(2023, 5, 1)
end_date = start_date.replace(month=start_date.month%12 + 1)
image = SentinelGetter().get_image(start_date, end_date)
        
# tci_bands = ['TCI_R', 'TCI_G', 'TCI_B']
# tci_max = 0.02550000138580799
rgb_max = 3000
rgb_bands = ['B4', 'B3', 'B2']
visualization = {
    'min': 0.0,
    'max': rgb_max,
    'bands': rgb_bands,
}

m = geemap.Map()
center = (np.array(bbox.getInfo()['coordinates'][0][2]) + np.array(bbox.getInfo()['coordinates'][0][0]))/2
m.set_center(*center, 7)
m.addLayer(image.median().clip(bbox), visualization, 'RGB')
m

Map(center=[52.589238399995665, 9.173807602718487], controls=(WidgetControl(options=['position', 'transparent_…

array([ 9.1738076, 52.5892384])